In [50]:
import numpy as np
from math import *
import os
from os.path import join
import bpy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipyvolume as ipv
from numpy.linalg import norm

In [26]:
scale = 1
subdiv = 1
name = "mesh"
middle_point_cache = {}

In [27]:
def vertex(x, y, z):
    """ Return vertex coordinates fixed to the unit sphere """
    length = np.sqrt(x**2 + y**2 + z**2)
    return [(i * scale) / length for i in (x,y,z)]

In [28]:
def middle_point(point_1, point_2):
    """ Find a middle point and project to the unit sphere """
    smaller_index = min(point_1, point_2)
    greater_index = max(point_1, point_2)
    
    key = "{0}-{1}".format(smaller_index, greater_index)
    
    if key in middle_point_cache:
        return middle_point_cache[key]
    
    # If it's not in cache, then we can cut it
    vert_1 = verts[point_1]
    vert_2 = verts[point_2]
    middle = [sum(i)/2 for i in zip(vert_1, vert_2)]
    
    verts.append(vertex(*middle))
    
    index = len(verts) - 1
    middle_point_cache[key] = index
    
    return index

In [29]:
PHI = (1 + np.sqrt(5)) / 2

In [30]:
verts = [ vertex(-1, PHI, 0), 
         vertex( 1, PHI, 0), 
         vertex(-1, -PHI, 0), 
         vertex( 1, -PHI, 0), 
         vertex(0, -1, PHI), 
         vertex(0, 1, PHI), 
         vertex(0, -1, -PHI), 
         vertex(0, 1, -PHI), 
         vertex( PHI, 0, -1), 
         vertex( PHI, 0, 1), 
         vertex(-PHI, 0, -1), 
         vertex(-PHI, 0, 1), ]

In [31]:
faces = [ 
    # 5 faces around point 0 
    [0, 11, 5], 
    [0, 5, 1], 
    [0, 1, 7], 
    [0, 7, 10], 
    [0, 10, 11], 
    # Adjacent faces 
    [1, 5, 9], 
    [5, 11, 4], 
    [11, 10, 2], 
    [10, 7, 6], 
    [7, 1, 8], 
    # 5 faces around 3 
    [3, 9, 4], 
    [3, 4, 2], 
    [3, 2, 6], 
    [3, 6, 8], 
    [3, 8, 9], 
    # Adjacent faces 
    [4, 9, 5], 
    [2, 4, 11], 
    [6, 2, 10], 
    [8, 6, 7], 
    [9, 8, 1], 
]

In [34]:
for i in range(subdiv): 
    faces_subdiv = [] 
    for tri in faces: 
        v1 = middle_point(tri[0], tri[1]) 
        v2 = middle_point(tri[1], tri[2]) 
        v3 = middle_point(tri[2], tri[0]) 
        faces_subdiv.append([tri[0], v1, v3]) 
        faces_subdiv.append([tri[1], v2, v1]) 
        faces_subdiv.append([tri[2], v3, v2]) 
        faces_subdiv.append([v1, v2, v3]) 
    faces = faces_subdiv

In [38]:
x_axis = np.array([point[0] for point in verts])
y_axis = np.array([point[1] for point in verts])
z_axis = np.array([point[2] for point in verts])

In [39]:
fig = ipv.figure()
ipv.scatter(x_axis, y_axis, z_axis, marker="sphere")
ipv.show()
ipv.save("plt.html")

In [40]:
def euclidean_distance(p1, p2):
    return np.sqrt(sum(np.square(np.array(list(map(lambda x: x[0] - x[1], zip(p1, p2)))))))

In [41]:
def great_circle_distance(r, p1, p2):
    """ Calculate the great circle distance by vector """
    p1, p2 = np.array(p1), np.array(p2)
    dist = np.arccos(np.matmul(p1, p2)) * r
    return dist

In [42]:
def find_min_dist(verts):
    min_dist = np.inf
    point = verts[0]
    for p in verts[1:]:
        dist = great_circle_distance(scale, point, p)
        min_dist = min(min_dist, dist)
    return min_dist

In [43]:
def find_neibors(verts, min_dist):
    neibors = []
    point = verts[0]
    for p in verts[1:]:
        dist = great_circle_distance(scale, point, p)
        if abs(dist - min_dist) <= 0.001:
            neibors.append(p)
    neibors.append(point)
    return neibors

In [61]:
def get_radius(scale, verts):
    min_dist = find_min_dist(verts)
    d = 0.5 * min_dist / np.cos(np.pi / 6) 
    
    x = verts[5]
    
    v = np.random.rand(3)
    v = v - np.matmul(v, x) * x
    v = v / norm(v) * d

    p = np.cos(norm(v)) * x + np.sin(norm(v)) * v / norm(v)
    
    origin = np.array([0, 0, 0])
    center = 0.5 * (origin + x)
    
    radius = euclidean_distance(p, center)
    
    return radius

In [63]:
verts = np.array(verts)

In [64]:
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)

In [76]:
ipv.figure()
R = get_radius(scale, verts)
R = 0.562
origin = np.array([0, 0, 0])
for vert in verts:
    center = 0.5 * (origin + vert)
    x = R * np.outer(np.cos(u), np.sin(v)) + center[0]
    y = R * np.outer(np.sin(u), np.sin(v))  + center[1]
    z = R * np.outer(np.ones(np.size(u)), np.cos(v)) + center[2]
    ipv.plot_mesh(x, y, z, u=y, v=x, wireframe=False, color="red")
    
x = scale * np.outer(np.cos(u), np.sin(v))
y = scale * np.outer(np.sin(u), np.sin(v))
z = scale * np.outer(np.ones(np.size(u)), np.cos(v)) 
ipv.plot_mesh(x, y, z, u=y, v=x, wireframe=False, color="blue")
ipv.show() 

In [72]:
"icosahedral grid"

'icosahedral grid'

In [49]:
scale

1

0.5483072243203564